In [177]:
import os
import rasterio
import rasterio.mask
from rasterio.merge import merge
import geopandas as gpd

In [182]:
def rasters(path):
    path_rasters = []
    for root, directory, files in os.walk(path):
        for file in files:
            path_rasters.append(os.path.join(root,file))
            
    return path_rasters

def get_bounding(dist_crop, indice):
    gdf = gpd.read_file('./footprints.gpkg',driver="GPKG")
    filter = gdf["indice"] == int(indice)
            
    return gdf[filter].buffer(dist_crop).head(1)

def crop_export(dist_crop,bits):
    path_rasters = rasters('./rasters')
    
    for raster_path in path_rasters:
        indice = raster_path.split('/')[-1].split('_')[0] #split para retirar o numero indice do titulo da imagem
        bounding = get_bounding(dist_crop, indice)

        with rasterio.open(raster_path, 'r') as dst:
            out_image, out_transform = rasterio.mask.mask(dst, bounding, crop=False)
            prof = dst.profile
            
        with rasterio.open(f'./output/cropped_{indice}.tif',"w", **prof) as dest:
            dest.write(out_image.astype(bits))

def create_filestomosaic(path):
    images_to_mosaic = rasters('./output')

    src_files=[]
 
    for fp in images_to_mosaic:
        src = rasterio.open(fp)
        src_files.append(src)
        out_meta = src.meta.copy()

    return out_meta, src.crs, src_files            
            
def gen_mosaic(bits):
    
    out_meta, crs, src_files_to_mosaic = create_filestomosaic('./output/')
    mosaic, out_trans = merge(src_files_to_mosaic, dtype=bits)

    return crs, out_meta, mosaic, out_trans
            
    
def create_mosaic(bits):
    
    if bits == 8:
        bits = rasterio.uint8
    else:
        bits = rasterio.uint16
    
    crop_export(-300, bits)
    
    crs, out_meta, mosaic, out_trans = gen_mosaic(bits)
    out_meta.update({"driver": "GTiff",
                     "height": mosaic.shape[1],
                     "width": mosaic.shape[2],
                     "transform": out_trans,
                     "dtype": bits,
                     "compress": 'LZW',
                     "crs": crs
                     }
                    )

    with rasterio.open(
            f'./output/rasterio.tif',
            "w", **out_meta) as dest:
        dest.write(mosaic.astype(bits))

    return

if __name__== '__main__':
    create_mosaic(8)
    

SyntaxError: can't use starred expression here (1411769123.py, line 4)